In [1]:
import pandas as pd
import numpy as np
import re
import time
import ast
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.model_selection import KFold
from sklearn.metrics import log_loss



In [2]:
nameTrainCSV = 'trainWithListOfCleanWords'
nameTestCSV = 'testWithListOfCleanWords'

train = pd.read_csv('../data/processed/' + nameTrainCSV + '.csv', encoding='utf-8')
train['BagOfWords'] = dict
train.head(10)


,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,cleanWordsAsText,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"['explanation', 'edits', 'made', 'username', '...",explanation edits made username hardcore metal...,<class 'dict'>
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"['aww', 'match', 'background', 'colour', 'seem...",aww match background colour seemingly stuck th...,<class 'dict'>
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"['hey', 'man', 'really', 'trying', 'edit', 'wa...",hey man really trying edit war guy constantly ...,<class 'dict'>
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"['make', 'real', 'suggestion', 'improvement', ...",make real suggestion improvement wondered sect...,<class 'dict'>
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"['sir', 'hero', 'chance', 'remember', 'page']",sir hero chance remember page,<class 'dict'>
5,5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,"['congratulation', 'well', 'use', 'tool', 'wel...",congratulation well use tool well talk,<class 'dict'>
6,6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,"['cocksucker', 'piss', 'around', 'work']",cocksucker piss around work,<class 'dict'>
7,7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,"['vandalism', 'matt', 'shirvington', 'article'...",vandalism matt shirvington article reverted pl...,<class 'dict'>
8,8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,"['sorry', 'word', 'nonsense', 'offensive', 'an...",sorry word nonsense offensive anyway intending...,<class 'dict'>
9,9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,"['alignment', 'subject', 'contrary', 'dulithgow']",alignment subject contrary dulithgow,<class 'dict'>


In [3]:
start = time.time()
for x in range(len(train)):
    train.set_value(col='listOfCleanWords',
                index=x,
                value=ast.literal_eval(train["listOfCleanWords"][x]))
    train.set_value(col='BagOfWords',
                index=x,
                value=Counter(train["listOfCleanWords"][x]))
end = time.time()
print(end - start)

21.50680136680603


In [4]:
train.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,cleanWordsAsText,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[explanation, edits, made, username, hardcore,...",explanation edits made username hardcore metal...,"{'explanation': 1, 'edits': 1, 'made': 1, 'use..."
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[aww, match, background, colour, seemingly, st...",aww match background colour seemingly stuck th...,"{'aww': 1, 'match': 1, 'background': 1, 'colou..."
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, really, trying, edit, war, guy, con...",hey man really trying edit war guy constantly ...,"{'hey': 1, 'man': 1, 'really': 1, 'trying': 1,..."
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[make, real, suggestion, improvement, wondered...",make real suggestion improvement wondered sect...,"{'make': 1, 'real': 1, 'suggestion': 1, 'impro..."
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page]",sir hero chance remember page,"{'sir': 1, 'hero': 1, 'chance': 1, 'remember':..."


In [5]:
# MULTICLASS PREDICTION

In [6]:
classLabel = {
    0: "neutral",
    1: "toxic",
    2 : "severe_toxic",
    3 : "obscene",
    4 : "threat",
    5 : "insult",
    6 : "identity_hate" 
}


In [7]:
y = np.empty((len(train['cleanWordsAsText']),),dtype=object)
allTextToxicTrain = dict()
for idx in classLabel:
    if classLabel[idx] != "neutral":
        T = np.where(train[classLabel[idx]] == 1)[0]
        allTextToxicTrain[idx] = T
        for i in T:
            if y[i] is None:
                y[i] = [idx]                
            else:
                y[i].append(idx)
indxsOfNeutralTexts = np.where(y == None) 
y[indxsOfNeutralTexts] = [[0]]
indxsOfNeutralTexts = indxsOfNeutralTexts[0]

allTextsNoToxicTrain = [str(train['cleanWordsAsText'][x]) for x in indxsOfNeutralTexts]

for i in allTextToxicTrain.keys():
    allTextToxicTrain[i] = [str(train['cleanWordsAsText'][j]) for j in allTextToxicTrain[i]]



In [8]:
###### GET THE BEST WORDS IN ALL CORPUS #####

topNFeatures = 1000
Nfolds = 5

#Neutral data

tdidfVectTrainNoToxic = TfidfVectorizer(max_features=topNFeatures)
matrixTdidfTrainNoToxic = tdidfVectTrainNoToxic.fit_transform(allTextsNoToxicTrain)

featureArrayNoToxicTrain = np.array(tdidfVectTrainNoToxic.get_feature_names())
tfidfOrderedNoToxicTrain = np.argsort(matrixTdidfTrainNoToxic.toarray()).flatten()[::-1]
selectedWordsTrainNoToxic = featureArrayNoToxicTrain[tfidfOrderedNoToxicTrain][:topNFeatures]


#Toxic data
selectedWordsTrainToxic = dict()
selectedWordsTrain = []
for idx in allTextToxicTrain.keys():
    tdidfVectTrainToxic = TfidfVectorizer(max_features=topNFeatures)
    matrixTdidfTrainToxic = tdidfVectTrainToxic.fit_transform(allTextToxicTrain[idx])
    featureArrayToxicTrain = np.array(tdidfVectTrainToxic.get_feature_names())
    tfidfOrderedToxicTrain = np.argsort(matrixTdidfTrainToxic.toarray()).flatten()[::-1]
    selectedWordsTrainToxic[idx] = featureArrayToxicTrain[tfidfOrderedToxicTrain][:topNFeatures]
# Get words
for x in range(topNFeatures):
    if len(selectedWordsTrain) <= topNFeatures: 
        words = [selectedWordsTrainToxic[idx][x] for idx in selectedWordsTrainToxic.keys()] + [selectedWordsTrainNoToxic[x]]
        selectedWordsTrain = np.unique(np.append(selectedWordsTrain, np.unique(words)))    
selectedWordsTrain = selectedWordsTrain[:topNFeatures]


In [9]:
allTrainText = [txt if txt is not np.nan else '' for txt in train['cleanWordsAsText']]

X = allTrainText
yBinary = MultiLabelBinarizer().fit_transform(y)

kf = KFold(n_splits=Nfolds, random_state=True)
kf.get_n_splits(allTrainText)

5

In [10]:
# PREDICCION MULTICLASS Linear SVC

meanAcc = 0.0
meanLogLoss = 0.0
meanFmeasure = 0.0
for train_index, test_index in kf.split(X):
    X_train = [X[i] for i in train_index]
    X_test = [X[i] for i in test_index]
    y_train, y_test = yBinary[train_index], yBinary[test_index]

    classifier = Pipeline([
    ('vectorizer', CountVectorizer(vocabulary=selectedWordsTrain)),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(LinearSVC()))])
    
    classifier.fit(X_train, y_train)
    predicted = classifier.predict(X_test)
    acc = accuracy_score(y_test, predicted)
    fmeausre = f1_score(y_test, predicted, labels=[0,1,2,3,4,5,6], average=None)
    logloss = log_loss(y_pred=predicted, y_true=y_test)
    print("Acc: " + str(acc))
    print("Logloss: " + str(logloss))
    print("Fmeausre: " + str(fmeausre))
    meanAcc += acc
    meanLogLoss += logloss
    meanFmeasure += fmeausre
meanAcc = meanAcc / Nfolds
meanLogLoss = meanLogLoss / Nfolds
meanFmeasure = meanFmeasure / Nfolds
print("\n Linear SVC")
print("La precisión media para " + str(Nfolds) + " folds es de: " + str(round(meanAcc, 5)))
print("Media Logloss: " + str(round(meanLogLoss, 5)))
print("Media Fmeasure: " + str(meanFmeasure))

Acc: 0.911859627135
Logloss: 3.64682518505
Fmeausre: [ 0.97313433  0.7011141   0.18137255  0.76882079  0.10169492  0.62592593
  0.29378531]
Acc: 0.913924923231
Logloss: 3.55244151581
Fmeausre: [ 0.9739479   0.70452771  0.22102426  0.76571807  0.10769231  0.62758362
  0.25072886]
Acc: 0.915554302187
Logloss: 3.63885730786
Fmeausre: [ 0.97335592  0.69835035  0.20151134  0.7752809   0.11428571  0.64002987
  0.29268293]
Acc: 0.914394936392
Logloss: 3.45078305765
Fmeausre: [ 0.97392853  0.70946341  0.21        0.77880184  0.27184466  0.64937993
  0.28571429]
Acc: 0.915648304819
Logloss: 3.57172740691
Fmeausre: [ 0.9738392   0.70112006  0.18617021  0.77176471  0.26315789  0.6478979
  0.16184971]

 Linear SVC
La precisión media para 5 folds es de: 0.91428
Media Logloss: 3.57213
Media Fmeasure: [ 0.97364118  0.70291513  0.20001567  0.77207726  0.1717351   0.63816345
  0.25695222]


In [11]:
# PREDICCION MULTICLASS Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
meanAcc = 0.0
meanLogLoss = 0.0
meanFmeasure = 0.0
for train_index, test_index in kf.split(X):
    X_train = [X[i] for i in train_index]
    X_test = [X[i] for i in test_index]
    y_train, y_test = yBinary[train_index], yBinary[test_index]

    classifier = Pipeline([
    ('vectorizer', CountVectorizer(vocabulary=selectedWordsTrain)),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(DecisionTreeClassifier()))])
    
    classifier.fit(X_train, y_train)
    predicted = classifier.predict(X_test)
    acc = accuracy_score(y_test, predicted)
    fmeausre = f1_score(y_test, predicted, labels=[0,1,2,3,4,5,6], average=None)
    logloss = log_loss(y_pred=predicted, y_true=y_test)
    print("Acc: " + str(acc))
    print("Logloss: " + str(logloss))
    print("Fmeausre: " + str(fmeausre))
    meanAcc += acc
    meanLogLoss += logloss
    meanFmeasure += fmeausre
meanAcc = meanAcc / Nfolds
meanLogLoss = meanLogLoss / Nfolds
meanFmeasure = meanFmeasure / Nfolds
print("\n Decission tree")
print("La precisión media para " + str(Nfolds) + " folds es de: " + str(round(meanAcc, 5)))
print("Media Logloss: " + str(round(meanLogLoss, 5)))
print("Media Fmeasure: " + str(meanFmeasure))

Acc: 0.870938430205
Logloss: 3.88108457188
Fmeausre: [ 0.96153245  0.62897405  0.28571429  0.72682927  0.2345679   0.55902664
  0.25933202]
Acc: 0.874099141443
Logloss: 3.80541251501
Fmeausre: [ 0.9622036   0.63221361  0.27238806  0.73442826  0.28877005  0.56719368
  0.34016393]
Acc: 0.876543209877
Logloss: 3.7292731851
Fmeausre: [ 0.96379707  0.63430992  0.27686703  0.72543618  0.1986755   0.59016393
  0.34033613]
Acc: 0.875227173028
Logloss: 3.66247038632
Fmeausre: [ 0.96388047  0.64148973  0.26497278  0.73427626  0.28947368  0.58887015
  0.3254902 ]
Acc: 0.873723130914
Logloss: 3.81864815408
Fmeausre: [ 0.96253962  0.6302565   0.29181495  0.72064277  0.1875      0.56849768
  0.28937729]

 Decission tree
La precisión media para 5 folds es de: 0.87411
Media Logloss: 3.77938
Media Fmeasure: [ 0.96279064  0.63344876  0.27835142  0.72832255  0.23979743  0.57475042
  0.31093992]


In [12]:
# PREDICCION MULTICLASS SGD
from sklearn import linear_model
meanAcc = 0.0
meanLogLoss = 0.0
meanFmeasure = 0.0
for train_index, test_index in kf.split(X):
    X_train = [X[i] for i in train_index]
    X_test = [X[i] for i in test_index]
    y_train, y_test = yBinary[train_index], yBinary[test_index]

    classifier = Pipeline([
    ('vectorizer', CountVectorizer(vocabulary=selectedWordsTrain)),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(linear_model.SGDClassifier()))])
    
    classifier.fit(X_train, y_train)
    predicted = classifier.predict(X_test)
    acc = accuracy_score(y_test, predicted)
    fmeausre = f1_score(y_test, predicted, labels=[0,1,2,3,4,5,6], average=None)
    logloss = log_loss(y_pred=predicted, y_true=y_test)
    print("Acc: " + str(acc))
    print("Logloss: " + str(logloss))
    print("Fmeausre: " + str(fmeausre))
    meanAcc += acc
    meanLogLoss += logloss
    meanFmeasure += fmeausre
meanAcc = meanAcc / Nfolds
meanLogLoss = meanLogLoss / Nfolds
meanFmeasure = meanFmeasure / Nfolds
print("\n SGD")
print("La precisión media para " + str(Nfolds) + " folds es de: " + str(round(meanAcc, 5)))
print("Media Logloss: " + str(round(meanLogLoss, 5)))
print("Media Fmeasure: " + str(meanFmeasure))

C:\Anaconda2\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Acc: 0.910104966317
Logloss: 4.13318245579
Fmeausre: [ 0.9696784   0.63508698  0.          0.74441004  0.          0.57109191
  0.17777778]


C:\Anaconda2\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Acc: 0.911950867958
Logloss: 4.05820093707
Fmeausre: [ 0.97011549  0.63927518  0.          0.73850377  0.          0.58762887
  0.1483871 ]


C:\Anaconda2\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Acc: 0.913674249546
Logloss: 4.00252631066
Fmeausre: [ 0.97064726  0.64115035  0.          0.75118644  0.          0.59797823
  0.19444444]


C:\Anaconda2\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Acc: 0.913580246914
Logloss: 3.91018451727
Fmeausre: [ 0.97155689  0.65601346  0.          0.76474576  0.          0.59457302
  0.16049383]
Acc: 0.913329573228
Logloss: 4.07076525803
Fmeausre: [ 0.97039328  0.63679044  0.          0.74644468  0.          0.59232614
  0.11480363]

 SGD
La precisión media para 5 folds es de: 0.91253
Media Logloss: 4.03497
Media Fmeasure: [ 0.97047826  0.64166328  0.          0.74905814  0.          0.58871963
  0.15918135]


C:\Anaconda2\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
# PREDICCION MULTICLASS NB
from sklearn.naive_bayes import MultinomialNB
meanAcc = 0.0
meanLogLoss = 0.0
meanFmeasure = 0.0
for train_index, test_index in kf.split(X):
    X_train = [X[i] for i in train_index]
    X_test = [X[i] for i in test_index]
    y_train, y_test = yBinary[train_index], yBinary[test_index]

    classifier = Pipeline([
    ('vectorizer', CountVectorizer(vocabulary=selectedWordsTrain)),
    ('tfidf', TfidfTransformer()),
    ('clf', OneVsRestClassifier(MultinomialNB()))])
    
    classifier.fit(X_train, y_train)
    predicted = classifier.predict(X_test)
    acc = accuracy_score(y_test, predicted)
    fmeausre = f1_score(y_test, predicted, labels=[0,1,2,3,4,5,6], average=None)
    logloss = log_loss(y_pred=predicted, y_true=y_test)
    print("Acc: " + str(acc))
    print("Logloss: " + str(logloss))
    print("Fmeausre: " + str(fmeausre))
    meanAcc += acc
    meanLogLoss += logloss
    meanFmeasure += fmeausre
meanAcc = meanAcc / Nfolds
meanLogLoss = meanLogLoss / Nfolds
meanFmeasure = meanFmeasure / Nfolds
print("\n NB")
print("La precisión media para " + str(Nfolds) + " folds es de: " + str(round(meanAcc, 5)))
print("Media Logloss: " + str(round(meanLogLoss, 5)))
print("Media Fmeasure: " + str(meanFmeasure))

Acc: 0.909634967883
Logloss: 4.13693716919
Fmeausre: [ 0.96985472  0.63666947  0.28853755  0.70264397  0.07207207  0.56593186
  0.26558266]
Acc: 0.911480854797
Logloss: 4.05510293663
Fmeausre: [ 0.97024962  0.63562232  0.37280702  0.69612459  0.064       0.58431532
  0.23098592]
Acc: 0.913204236385
Logloss: 4.07422282062
Fmeausre: [ 0.97072252  0.63830706  0.35123967  0.69888208  0.02105263  0.58092949
  0.25671642]
Acc: 0.912326878486
Logloss: 3.98995978953
Fmeausre: [ 0.97085796  0.644621    0.32704403  0.69952881  0.06896552  0.58651503
  0.24861878]
Acc: 0.913266904807
Logloss: 4.04399428126
Fmeausre: [ 0.97101253  0.64274908  0.36363636  0.70246414  0.11881188  0.57890499
  0.15300546]

 NB
La precisión media para 5 folds es de: 0.91198
Media Logloss: 4.06004
Media Fmeasure: [ 0.97053947  0.63959379  0.34065293  0.69992872  0.06898042  0.57931934
  0.23098185]
